In [1]:
import numpy as np
import skimage as sk
import skimage.io as skio
import json
import copy
import triangle
import matplotlib.pyplot as plt
import skimage as ski
from PIL import Image
import imageio
import os
from tqdm import tqdm
import cv2
from scipy import signal
import numpy as np

In [2]:
depth = skio.imread("example_depth.png")
  
depth = sk.img_as_float(depth)[:,:,0]

In [3]:
colors = skio.imread("example_img.png")
  
colors = sk.img_as_float(colors)
colors = np.array(colors[:,:,:3])

In [4]:
# skio.imshow(colors)

In [5]:
print(depth.shape)

(256, 512)


In [6]:
h = 256
w = 512

In [8]:
def calculate_cart_coordinate(depth):
    points = []
    selected_index = []
    for i in range(h):
        for j in range(w):
            phi = (w-j-1)/(w-1) * 2* 3.14159 + 0.5*3.14159
            theta = (h-i-1)/(h-1) * 3.14159
            
#             phi = (w-i)/(w) * 2* 3.14159
#             theta = (1-(h-j)/(h)) * 3.14159
            v = np.array([np.sin(theta)*np.cos(phi), np.sin(theta)*np.sin(phi), np.cos(theta)])
#             points.append(copy.copy(v*depth[j,i]*1000))
            if depth[i,j] != 1:
                points.append(copy.copy(v*depth[i,j]*255))
                
                selected_index.append(i*w + j)
    return points, selected_index
            

In [9]:
points, selected_index = calculate_cart_coordinate(depth)

In [13]:
print(colors.shape)

(256, 512, 3)


In [15]:
colors = colors.reshape(-1,3)

In [16]:
colors = colors[selected_index]

In [17]:
import open3d as o3d
import numpy as np

# Define a list of points (replace this with your own coordinates)
points = np.array(points)
# Create a point cloud
point_cloud = o3d.geometry.PointCloud()
point_cloud.points = o3d.utility.Vector3dVector(points)
point_cloud.colors = o3d.utility.Vector3dVector(colors)

# Visualize the point cloud
o3d.visualization.draw_geometries([point_cloud])

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
